In [2]:
# Custom exception class to give clean, consistent error reporting.

import os  ## not required here
import sys  # Needed to access exception info like line numbers and filenames    
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
class TradingBotException(Exception):
    def __init__(self):
        print("this message from init")
        
    def __str__(self):
        print("this message from str")
        return ""
    
    def __repr__(self):
        print("this message from repr")
        return ""

In [4]:
obj = TradingBotException()

this message from init


In [5]:

print(obj)

this message from str



In [6]:
obj

this message from repr


In [7]:
class TradingBotException(Exception):  # Inherit from Python's built-in Exception class
    def __init__(self,error_message,error_details_using_sys:sys):
        self.error_message = error_message  # Store the original error message

         # Extract traceback details using sys module to get line number and filename
        _,_,exc_tb = error_details_using_sys.exc_info()
        
        # Capture the line number where the exception occurred
        self.lineno=exc_tb.tb_lineno

         # Capture the file name where the exception occurred
        self.file_name=exc_tb.tb_frame.f_code.co_filename 
    
    def __str__(self):
        
        # Custom string representation for logging/debugging
        return "Error occured in python script name [{0}] line number [{1}] error message [{2}]".format(
        self.file_name, self.lineno, str(self.error_message))

In [8]:

def test(message):
    print("add"+message)
    print(sys.exc_info())
    #print(sys.exc_info())

In [9]:
test("hi hi hi")

addhi hi hi
(None, None, None)


In [10]:
try:
    test(2)
except Exception as e:
    raise TradingBotException(e,sys)

TradingBotException: Error occured in python script name [C:\Users\Asus\AppData\Local\Temp\ipykernel_36120\638354084.py] line number [2] error message [can only concatenate str (not "int") to str]

In [11]:
# if __name__ == '__main__':
try:
    a=1/0  # This will raise a ZeroDivisionError
    print("This will not be printed",a)
except Exception as e:
    raise TradingBotException(e,sys)  # Converts into your custom error with context

TradingBotException: Error occured in python script name [C:\Users\Asus\AppData\Local\Temp\ipykernel_36120\2584503402.py] line number [3] error message [division by zero]

In [12]:
import logging
import os
from datetime import datetime


In [13]:
LOG_FILE = f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"  ## Log File

In [14]:

logs_path = os.path.join(os.getcwd(), "logs")  # create log path/folder

In [15]:
os.makedirs(logs_path, exist_ok=True)   ## made a directory

In [16]:

LOG_FILE_PATH = os.path.join(logs_path, LOG_FILE)

In [17]:
logger=logging.getLogger("my_agentic_app")

In [18]:

logging.basicConfig(
    filename=LOG_FILE_PATH,
    format="[ %(asctime)s ] %(lineno)d %(name)s - %(levelname)s - %(message)s",
    level=logging.INFO,
)

In [19]:

logger.info("this is my testing message")

In [20]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [21]:
len(embeddings.embed_query("hi my name is sunny"))

768

In [ ]:
384, 768,1024

In [22]:
from langchain_groq import ChatGroq

In [23]:
groq_model=ChatGroq(model="deepseek-r1-distill-llama-70b",api_key="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")
print(groq_model.invoke("hi"))

AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [ ]:
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import ToolNode, tools_condition
#from langchain_core.messages import AIMessage, HumanMessage
from typing_extensions import Annotated, TypedDict
from utils.model_loaders import ModelLoader
from toolkit.tools import *

In [ ]:
from IPython.display import Image,display

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
class GraphBuilder:
    def __init__(self):
        self.model_loader=ModelLoader()
        self.llm = self.model_loader.load_llm()
        self.tools = [retriever_tool, financials_tool, tavilytool]
        llm_with_tools = self.llm.bind_tools(tools=self.tools)
        self.llm_with_tools = llm_with_tools
        self.graph = None
    
    def _chatbot_node(self,state:State):
         return {"messages": [self.llm_with_tools.invoke(state["messages"])]}

    def build(self):
        graph_builder = StateGraph(State)
        
        graph_builder.add_node("chatbot", self._chatbot_node)
        
        tool_node=ToolNode(tools=self.tools)
        graph_builder.add_node("tools", tool_node)
        
        graph_builder.add_conditional_edges("chatbot", tools_condition)
        graph_builder.add_edge("tools", "chatbot")
        graph_builder.add_edge(START, "chatbot")
        
        self.graph = graph_builder.compile()
    
    def visulize_graph(self):
        display(Image(self.graph.get_graph().draw_mermaid_png()))
        
    def get_graph(self):
        if self.graph is None:
            raise ValueError("Graph not built. Call build() first.")
        return self.graph

In [ ]:
graph_service = GraphBuilder()

In [ ]:
graph_service.build()

In [ ]:
graph = graph_service.get_graph()

In [ ]:
graph_service.visulize_graph()

In [ ]:
# Assuming request is a pydantic object like: {"question": "your text"}
messages={"messages": "can you give me a deep knowledge of the stock market?"}

In [ ]:
result = graph.invoke(messages)

In [ ]:
result

In [ ]:
result = graph.invoke(messages)

In [ ]:
result

In [ ]:
result = graph.invoke(messages)

In [ ]:
result